In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import os
from itertools import izip
import itertools
from collections import defaultdict
from gensim.parsing import preprocessing as gprocessing
from spacy.en import English as nlp
from bs4 import BeautifulSoup

In [3]:
os.chdir("..")
os.getcwd()

'/mnt/Storage/Coding_Projects/Candidate_Classifier'

In [7]:
with open('candidate_classifier/data/R_Debate_2015-08-06.txt', 'r') as _f:
    test = _f.read()

In [8]:
p = re.compile(r"(^\[?[A-Z ]*?\]?:)", re.U|re.M)

In [13]:
chunks = re.split(p, test, maxsplit=1)

In [14]:
chunks[1:]

['KELLY:',
 ' Welcome to the first debate night of the 2016 presidential campaign, live from Quicken Loans Arena in Cleveland, Ohio.\n\nI\'m Megyn Kelly... [applause]... along with my co-moderators, Brett Baier and Chris Wallace.\n\nTonight... [applause] Nice.\n\nTonight, thousands of people here in the Q, along with millions of voters at home will get their very first chance to see the candidates face off in a debate, answering the questions you want answered.\n\nBAIER: Less than a year from now, in this very arena, one of these 10 candidates or one of the seven on the previous debate tonight will accept the Republican party\'s nomination. [applause]\n\nTonight\'s candidates were selected based on an average of five national polls. Just a few hours ago, you heard from the candidates ranked 11th through 17. And now, the prime-time event, the top 10.\n\nWALLACE: Also of note, Fox News is partnering for tonight\'s debate with Facebook. For the past several weeks, we\'ve been asking you f

In [7]:
def pairwise(iterable):
    """http://stackoverflow.com/questions/5389507/iterating-over-every-two-elements-in-a-list
    s -> (s0,s1), (s2,s3), (s4, s5), ..."""
    i = iter(iterable)
    return itertools.izip(i, i)

In [8]:
d = defaultdict(list)
for a,b in pairwise(chunks[1:]):
    d[a.strip(' :')].append(b)
    
d.keys()

['HUCKABEE',
 'PERRY',
 'BAIER',
 'TRUMP',
 'WALLACE',
 'FIORINA',
 'UNKNOWN',
 'QUESTION',
 'BUSH',
 'CHRISTIE',
 'CARSON',
 'WALKER',
 'RUBIO',
 'UNIDENTIFIED MALE',
 'CRUZ',
 'KASICH',
 'KELLY',
 '[UNKNOWN]',
 'PAUL',
 'MODERATOR',
 'UNIDENTIFIED FEMALE']

In [53]:
set(re.findall(r"\[[a-zA-Z ]*?\]", test))

{'[ catcalls]',
 '[UNKNOWN]',
 '[applause]',
 '[begin video clip]',
 '[booing]',
 '[cheering and applause]',
 '[close video clip]',
 '[commercial break]',
 '[crosstalk]',
 '[inaudible]',
 '[laught and applause]',
 '[laughter and applause]',
 '[laughter and boing]',
 '[laughter]',
 '[off mike]',
 '[timer sounds]',
 '[video clip begins]',
 '[video clip ends]'}

In [10]:
type(p)

_sre.SRE_Pattern

## String Processing Pipeline

In [9]:
from candidate_classifier.string_processing import *
from candidate_classifier import utils

In [38]:
# Replace [*] with ''
# Replace '. . .' with '...'
# Replace multiple ellipses with single ...
# Remove all sentences that end with ...

BRACKET_PATTERN = re.compile(r"\[[a-zA-Z ]*\]", re.U)
SPACED_ELLIPSIS_PATTERN = re.compile(r"((?:\.\s){3})")
MULTI_ELLIPSIS_PATTERN = re.compile(r"(?:(?:\.){3} ?)+")
ENDS_WITH_ELLIPSIS = lambda s: s[-3:] == '...'
STARTS_WITH_DASH = lambda s: s.strip()[0] == '-'

In [15]:
str_transformers = [StringTransformer(prefilter_substitutions=[BRACKET_PATTERN, 
                                                               'whitespace',
                                                               'strip',
                                                               (SPACED_ELLIPSIS_PATTERN, '...'),
                                                               (MULTI_ELLIPSIS_PATTERN, '...')])]

sent_transformers = [SentenceTransformer(filters=[ENDS_WITH_ELLIPSIS])]

In [16]:
s_processor = StringProcessor(str_transformers + sent_transformers)

In [17]:
processed = {k: [s_processor(s) for s in v] for k, v in d.iteritems()}

processed

{'BAIER': [u"Less than a year from now, in this very arena, one of these 10 candidates or one of the seven on the previous debate tonight will accept the Republican party's nomination. Tonight's candidates were selected based on an average of five national polls. Just a few hours ago, you heard from the candidates ranked 11th through 17. And now, the prime-time event, the top 10.",
  u'Neurosurgeon, Dr. Ben Carson. Texas Senator Ted Cruz. Florida Senator Marco Rubio.',
  u"It might be. It might be. We'll see.",
  u"It might be. The rules for tonight are simple. One minute for answers, 30 seconds for follow-ups. And if a candidate runs over, you'll hear this. Pleasant, no? We also have a big crowd here with us tonight in the home of the Cavaliers, as I mentioned. And while we expect them...we expect them to be enthusiastic, as you heard, we don't want to take anything away from the valuable time for the candidate. So, we're looking for somewhere between a reaction to a LeBron James dunk

In [36]:
with open('candidate_classifier/data/R_Debate_2015-08-06.txt', 'r') as _f:
    for i, l in enumerate(_f):
        if i > 10:
            break
        print re.match(p, l), l[:10]

<_sre.SRE_Match object at 0x7fa1bf44ab70> KELLY: Wel
None 

None I'm Megyn 
None 

None Tonight...
None 

None Tonight, t
None 

<_sre.SRE_Match object at 0x7fa1bf44ab70> BAIER: Les
None 

None Tonight's 


## Corpora

In [6]:
from candidate_classifier.nltk_model import NgramModel
from candidate_classifier import utils
from nltk.probability import LaplaceProbDist, LidstoneProbDist
from nltk.corpus import PlaintextCorpusReader
import os

In [7]:
from nltk.corpus import switchboard, nps_chat
from nltk.corpus.reader.util import *
from nltk.tokenize import *
import nltk

In [8]:
corpus_root = os.path.abspath('candidate_classifier/data')

In [136]:
#! /usr/bin/env python2

import os
import itertools

from candidate_classifier import utils
from nltk.corpus.reader.util import *
from nltk.corpus import PlaintextCorpusReader
import nltk
from nltk.tokenize import WordPunctTokenizer

__author__ = 'Eric Lind'


# TODO: Strip 'SPEAKER:' for words() and sents() methods
# TODO: define paras(), words(), and sents() that can take in speakers='foo'


class DebateCorpusReader(PlaintextCorpusReader):
    """
    Extension of the NLTK `PlaintextCorpusReader` designed to read
    presedential debate transcripts from The American Presidency Project
    http://www.presidency.ucsb.edu/debates.php
    """
    _speaker_pattern = re.compile(r"(^\[?[A-Z :\-?_]+\]?:+)", re.U | re.M)

    def __init__(self, root, fileids,
                 word_tokenizer=WordPunctTokenizer(),
                 sent_tokenizer=nltk.data.LazyLoader(
                     'tokenizers/punkt/english.pickle'),
                 encoding='utf8'):
        super(DebateCorpusReader, self).__init__(root,
                                                 fileids,
                                                 word_tokenizer=word_tokenizer,
                                                 sent_tokenizer=sent_tokenizer,
                                                 para_block_reader=self._make_para_block_reader(),
                                                 encoding=encoding)
        # Property cache
        self._speakers = {}
        # TODO: Cache speaker paras?

    def speakers(self, fileids=None):
        """Same as `speakers` but cleans cout common typos."""
        return list(set(s.strip(' :[]') for s in self.raw_speakers(fileids=fileids)))

    def raw_speakers(self, fileids=None):
        """
        Returns the names of all of the speakers in the given files.
        :return: A list of all of the unique speaker names.
        :rtype: list(str)
        """
        if fileids is None: fileids = self._fileids
        elif isinstance(fileids, string_types): fileids = [fileids]

        # FIXME: Cleanup
        if hasattr(fileids, '__getitem__'):
            # FIXME: confirm type is str[]
            f_key = '_'.join(fileids)
        else:
            f_key = fileids

        # Used cached value if it exists
        if f_key in self._speakers:
            return self._speakers[f_key]
        else:
            speakers = list(set(m.group(0) for m in self._speaker_pattern.finditer(self.raw(fileids=fileids))))
            self._speakers[f_key] = speakers

        return speakers


    # def raw(self, fileids=None, speakers=None):
    #     """
    #     :return: the given file(s) as a single string.
    #     :rtype: str
    #     """
    #     if fileids is None: fileids = self._fileids
    #     elif isinstance(fileids, string_types): fileids = [fileids]
    #
    #     if speakers is None: speakers = self.speakers(fileids)
    #     elif isinstance(speakers, string_types): speakers = [speakers]
    #
    #     return concat([self.open(f).read() for f in fileids])


    # TODO: Implement speakers
    def joined_speaker_paras(self, fileids=None, speakers=None):
        """Because of the nature of the text, interruptions, cross-talk, etc.
        many sentences are split across speaker boundaries.  This can cause unclean
        sentence tokenization.  This method is not particularly memory efficient
        because it reads in a while file at a time, but it avoids the tokenization
        issue.
        """
        if fileids is None: fileids = self._fileids
        elif isinstance(fileids, string_types): fileids = [fileids]

        if speakers is None:
            speakers = self.speakers(fileids=fileids)
        else:
            speakers = [s.upper() for s in speakers]

        paras = {}

        for s in speakers:
            reader = self._make_joined_reader(s)
            paras[s] = concat([self.CorpusView(path, reader, encoding=enc)
                               for (path, enc, fileid)
                               in self.abspaths(fileids, True, True)])

        return paras


    def _make_joined_reader(self, speaker):
        """
        Creates a reader function that takes in a stream from
        a debate transcript file and returns all of the utterances
        of the given speaker in that stream, joined as a single
        stream (without the speaker identifier).
        """
        start_re = self._make_speaker_pattern(speaker)
        end_re = self._speaker_pattern
        reader = self._make_para_block_reader(start_re=start_re, end_re=end_re)

        def new_reader(stream):
            acc = u''
            p = reader(stream)
            while p:
                acc += u' '.join(self._speaker_pattern.sub('', t, count=1) for t in p)
                p = reader(stream)
            return [self._word_tokenizer.tokenize(sent)
                    for sent in self._sent_tokenizer.tokenize(acc) if sent]

        return new_reader

    def all_speaker_paras(self, fileids=None):
        return {name: self.speaker_paras(name) for name in self.speakers(fileids=fileids)}

    def speaker_paras(self, speaker, fileids=None):
        """Returns a dictionary of
        {str speaker_name: CorpusView paragraphs}
        """
        speaker = speaker.upper()
        # TODO: Tokenize etc.
        reader = self._make_speaker_block_reader(speaker)

        return concat([self.CorpusView(path, reader, encoding=enc)
                       for (path, enc, fileid)
                       in self.abspaths(fileids, True, True)])

    def _make_speaker_block_reader(self, speaker):
        start_re = self._make_speaker_pattern(speaker)
        end_re = self._speaker_pattern
        reader = self._make_para_block_reader(start_re=start_re, end_re=end_re)

        def new_reader(stream):
            paras = []
            for para in reader(stream):
                para = self._speaker_pattern.sub('', para, count=1)
                paras.append([self._word_tokenizer.tokenize(sent)
                              for sent in self._sent_tokenizer.tokenize(para) if sent])
            return paras

        return new_reader

    def _read_word_block(self, stream):
        words = []
        for i in range(20):  # Read 20 lines at a time.
            # Replace any reference to the speakers
            line = self._speaker_pattern.sub('', stream.readline())
            words.extend(self._word_tokenizer.tokenize(line))
        return words

    def _read_para_block(self, stream):
        """Shadows parent method so that it can strip the leading 'SPEAKER:'."""
        paras = []
        for para in self._para_block_reader(stream):
            para = self._speaker_pattern.sub('', para, count=1)
            paras.append([self._word_tokenizer.tokenize(sent)
                          for sent in self._sent_tokenizer.tokenize(para) if sent])
        return paras


    def _make_para_block_reader(self, start_re=None, end_re=None):
        """Creates a closure for `read_regexp_block` with pre-defined regexp."""
        if start_re is None: start_re = self._speaker_pattern

        def pbr(stream):
            return read_regexp_block(stream, start_re, end_re=end_re)

        return pbr

    @staticmethod
    def _make_speaker_pattern(name):
        """
        Returns a compiled regular expression to match any speaker
        by name, even accounting for typos.
        """
        s = r"(^\[?{}[ :\-?_]*\]?:+)".format(re.escape(name))
        return re.compile(s, re.U | re.M)



class DummyTokenizer(object):
    def tokenize(self, s):
        return s

In [48]:
debates = DebateCorpusReader('candidate_classifier/data', '.*', word_tokenizer=DummyTokenizer())

In [49]:
debates.fileids()

['R_Debate_2015-08-06.txt',
 'R_Debate_2015-09-16.txt',
 'R_Debate_2015-10-28.txt',
 'R_Debate_2015-11-10.txt',
 'R_Debate_2015-12-15.txt',
 'R_Debate_2016-01-14.txt']

In [50]:
debates.speaker_paras('TRUMP')

[[u' I fully understand.\n\n'], [u' I fully understand.\n\n'], ...]

In [33]:
debates.all_speaker_paras()

{u'???': [[u' ... into bankruptcy.', u"That's what's wrong with politicians in Washington right now.", u'They think we can take a country into bankruptcy.', u'[crosstalk]\n\n']],
 u'AUDIENCE': [[u' Boo.\n\n'], [u' Boo.\n\n']],
 u'BAIER': [[u" Less than a year from now, in this very arena, one of these 10 candidates or one of the seven on the previous debate tonight will accept the Republican party's nomination.", u"[applause]\n\nTonight's candidates were selected based on an average of five national polls.", u'Just a few hours ago, you heard from the candidates ranked 11th through 17.', u'And now, the prime-time event, the top 10.\n\n'], [u' Neurosurgeon, Dr. Ben Carson.', u'[applause]\n\nTexas Senator Ted Cruz.', u'[applause]\n\nFlorida Senator Marco Rubio.', u'[applause]\n\n'], ...],
 u'BAKER': [[u' The candidates on stage tonight were selected based on their standing in an average of four national polls.', u'Those standings determining their position on the stage.', u'And here they 

In [23]:
debates.sents()

[[u'KELLY', u':', u'Welcome', u'to', u'the', u'first', u'debate', u'night', u'of', u'the', u'2016', u'presidential', u'campaign', u',', u'live', u'from', u'Quicken', u'Loans', u'Arena', u'in', u'Cleveland', u',', u'Ohio', u'.'], [u'I', u"'", u'm', u'Megyn', u'Kelly', u'...', u'[', u'applause', u']...', u'along', u'with', u'my', u'co', u'-', u'moderators', u',', u'Brett', u'Baier', u'and', u'Chris', u'Wallace', u'.'], ...]

In [156]:
debates.joined_speaker_paras()

{u'???': [u'', u" ... into bankruptcy. That's what's wrong with politicians in Washington right now. They think we can take a country into bankruptcy.\n\n[crosstalk]\n\n", ...],
 u'AUDIENCE': [u'', u'', u'', u'', u'', u' Boo.\n\n Boo.\n\n'],
 u'BAIER': [u' Less than a year from now, in this very arena, one of these 10 candidates or one of the seven on the previous debate tonight will accept the Republican party\'s nomination. [applause]\n\nTonight\'s candidates were selected based on an average of five national polls. Just a few hours ago, you heard from the candidates ranked 11th through 17. And now, the prime-time event, the top 10.\n\n Neurosurgeon, Dr. Ben Carson. [applause]\n\nTexas Senator Ted Cruz. [applause]\n\nFlorida Senator Marco Rubio. [applause]\n\n It might be. It might be. We\'ll see.\n\n It might be. The rules for tonight are simple. One minute for answers, 30 seconds for follow-ups. And if a candidate runs over, you\'ll hear this.\n\nPleasant, no?\n\nWe also have a big

## Model
To get more usable text, all paragraphs from a speaker are joined and munged and the tokenized afterwards.

In [7]:
from spacy.en import English
from candidate_classifier.string_processing import *

In [8]:
nlp = English(entity=False, load_vectors=False)

In [29]:
class TransformerWrapper(object):
    def __init__(self, transformer):
        self.transformer = transformer
    
    def tokenize(self, s):
        return self.transformer(s)

    
def sent_tokenizer(s):
    doc = nlp(s)
    return [u''.join(t.text_with_ws for t in sent) for sent in doc.sents]
#     return [sent for sent in doc.sents]

sent_tokenizer(u"This is a sentence. Here's another...")

[u'This is a sentence. ', u"Here's another..."]

In [122]:
def flatten(nested):
    try:
        try: nested + ''
        except TypeError: pass
        else: 
            # TODO: Make this more robust
            if not isinstance(nested, ConcatenatedCorpusView):
                raise TypeError
        for sublist in nested:
            for element in flatten(sublist):
                yield element
    except TypeError:
        yield nested

In [128]:
# Replace [*] with ''
# Replace '. . .' with '...'
# Replace multiple ellipses with single ...
# Remove all sentences that end with ...

BRACKET_PATTERN = re.compile(r"\[[a-zA-Z ]*\]", re.U)
SPACED_ELLIPSIS_PATTERN = re.compile(r"((?:\.\s){3})")
MULTI_ELLIPSIS_PATTERN = re.compile(r"(?:(?:\.){3} ?)+")
ENDS_WITH_ELLIPSIS = lambda s: s[-3:] == '...'
STARTS_WITH_ELLIPSIS = lambda s: s[:3] == '...'
STARTS_WITH_DASH = lambda s: s[0] == '-'
ENDS_WITH_DASH = lambda s: s[-1] == '-'


def add_delimiters(s):
    return '<S> ' + s + ' </S>'

In [176]:
doc_transformer = TransformerABC(
    prefilter_substitutions=[BRACKET_PATTERN,
                             (SPACED_ELLIPSIS_PATTERN, '...'),
                             (MULTI_ELLIPSIS_PATTERN, '...'),
                             'whitespace',
                             'strip',
                             'deaccent',
                             'lower'],
    tokenizer=sent_tokenizer)


sent_transformer = TransformerABC(
    prefilter_substitutions=['strip'],
    filters=[STARTS_WITH_ELLIPSIS, ENDS_WITH_ELLIPSIS, STARTS_WITH_DASH, ENDS_WITH_DASH],
    postfilter_substitutions=[add_delimiters])

In [177]:
debates = DebateCorpusReader('candidate_classifier/data', '.*', 
                             sent_tokenizer=TransformerWrapper(doc_transformer), 
                             word_tokenizer=TransformerWrapper(sent_transformer))

In [178]:
for t in flatten(debates.joined_speaker_paras()['???']):
    print t

<S> that's what's wrong with politicians in washington right now. </S>
<S> they think we can take a country into bankruptcy. </S>


In [163]:
len(debates.joined_speaker_paras()['???'])

3

In [166]:
def get_stats(sents):
    stats = {}
    
    words = list(flatten(sents))
    
    # Num words
    stats['num_words'] = len(words)
    # Num word_types
    stats['num_word_types'] = len(set(words))
    
    return stats

In [167]:
stats = {k: get_stats(v) for k,v in debates.joined_speaker_paras().iteritems()}
stats

{u'???': {'num_word_types': 20, 'num_words': 22},
 u'AUDIENCE': {'num_word_types': 3, 'num_words': 6},
 u'BAIER': {'num_word_types': 713, 'num_words': 1796},
 u'BAKER': {'num_word_types': 568, 'num_words': 1393},
 u'BARTIROMO': {'num_word_types': 941, 'num_words': 2914},
 u'BASH': {'num_word_types': 492, 'num_words': 1473},
 u'BLITZER': {'num_word_types': 746, 'num_words': 2528},
 u'BUSH': {'num_word_types': 2620, 'num_words': 14403},
 u'CARSON': {'num_word_types': 2075, 'num_words': 10258},
 u'CAVUTO': {'num_word_types': 1097, 'num_words': 3246},
 u'CHRISTIE': {'num_word_types': 2149, 'num_words': 11832},
 u'CRAMER': {'num_word_types': 95, 'num_words': 161},
 u'CRUZ': {'num_word_types': 2873, 'num_words': 13112},
 u'EPPERSON': {'num_word_types': 252, 'num_words': 498},
 u'FIORINA': {'num_word_types': 1988, 'num_words': 8775},
 u'HARWOOD': {'num_word_types': 594, 'num_words': 1591},
 u'HEWITT': {'num_word_types': 574, 'num_words': 1606},
 u'HUCKABEE': {'num_word_types': 1274, 'num_word

### MODELS!!!

In [168]:
from candidate_classifier.nltk_model import NgramModel
from nltk.probability import LaplaceProbDist, LidstoneProbDist
from nltk.corpus import PlaintextCorpusReader

In [204]:
# In order to have the models be better at starting and ending sentences, 
# it would be good to read in multi-sentence documents.  This function will take in
# an iterator of sentences and combine every N sentences into a single document

def chunker(it, n=10):
    args = [iter(it)] * n
    chunks = itertools.izip_longest(*args, fillvalue='')
    return chunks

In [203]:
for i, d in enumerate(chunker(flatten(debates.all_speaker_paras().values()[0]), 10)):
    if i > 5:
        break
    print d

(u'<S> dr. </S>', u'<S> carson, in recent weeks, a number of pharmaceutical companies has been accused of profiteering, for dramatically raising the prices of life-saving drugs. </S>', u'<S> you have spent a lifetime in medicine. </S>', u'<S> have these companies gone too far? </S>', u'<S> should the government be involved in controlling some of these price increases? </S>', u'<S> thank you, dr. </S>', u'<S> carson. </S>', u'<S> governor christie, there has been a lot of political rhetoric that some bank executives should have gone to jail for the 2008 financial crisis. </S>', u'<S> but general motors paid more than $1 billion in fines and settlements for its ignition switch defect. </S>', u'<S> one hundred and twenty- four people died as a result of these faulty switches. </S>')
(u'<S> no one went to jail. </S>', u'<S> as a former prosecutor, do you believe the people responsible for the switch and the cover-up belong behind bars? </S>', u'<S> thank you, governor christie. </S>', '', 

In [169]:
est = lambda freqdist, bins: LidstoneProbDist(freqdist, 0.2, bins)

In [219]:
models = {}

for k, v in debates.joined_speaker_paras().iteritems():
    if stats[k]['num_word_types'] >= 100:
#         docs = (concat(chunk) for chunk in itertools.islice((sent.split() for sent in flatten(v)), 10))
        docs = (' '.join(chunk).split() for chunk in chunker(flatten(v)))
#         print list(docs)[:2]
        models[k] = NgramModel(3, docs, estimator=est, bins=stats[k]['num_word_types'])

In [220]:
for i in range(5):
    print ' '.join(models['TRUMP'].generate(50)), '\n'

<S> so, look, we have a tremendous success. </S> <S> well, i'd like to respond, i'd like to tell you, we are spending $200 billion a year imbalance. </S> <S> assad is fighting isis. </S> <S> we cannot lose this election. </S> <S> but i say that i have used 

<S> second of all, let me — caesar's, the rolls-royce, as you too know, i know i built a very good people are going to find a finer men. </S> <S> but a lot — but the democrats are going to even say — i think, for me, nuclear is 

<S> i agree 100 percent, dana. </S> <S> mr. trump" — these are terrible people in actually 15 — going to have people come into our country has right now by people that want to endorse me. </S> <S> you feel safe right now? </S> <S> i spoke to a 

<S> it's hurting us from every standpoint. </S> <S> are you going to embarrass, but virtually every person that you wouldn't have millions of jobs and manufacturing back. </S> <S> one thing at a time. </S> <S> so we can get the senate, but i hope that answers your 

In [221]:
for i in range(5):
    print ' '.join(models['BUSH'].generate(50)), '\n'

<S> if they had — they are barrel bombing the innocents in that country. </S> <S> if we're serious about high growth, then we 'ought to create more government from the federal law. </S> <S> when i'm elected president, and hillary clinton, who can't even project force. </S> <S> to 

<S> yeah. here's — apart from that, there will be — this is recognizing that there will be — this is not a factor. </S> <S> everybody's record's going to win when we — you flew in with your 767, didn't you? </S> <S> imagine what it's like now to 

<S> it's a good job. </S> <S> that expertise needs to complete dialogue with us. </S> <S> so he supports clinton — — the pacific agreement. </S> <S> lay them out there. </S> <S> i just did. </S> <S> i look forward to talking tonight about how bad washington, d.c. 

<S> this would be devastating for agriculture and many of the hhs funding, there is a place where security can take place. </S> <S> 6.5 million people are suffering today in america. </S> <S> no it isn't

In [185]:
[sent.split() for sent in flatten(debates.joined_speaker_paras()['???'])]

[[u'<S>',
  u"that's",
  u"what's",
  u'wrong',
  u'with',
  u'politicians',
  u'in',
  u'washington',
  u'right',
  u'now.',
  u'</S>'],
 [u'<S>',
  u'they',
  u'think',
  u'we',
  u'can',
  u'take',
  u'a',
  u'country',
  u'into',
  u'bankruptcy.',
  u'</S>']]

In [16]:
reduce(lambda x, y: '{}({})'.format(y, x), ['a', 'b', 'c'], 'doc')

'c(b(a(doc)))'

# Scratch

In [77]:
from collections import deque

In [104]:
def rrun(s):
    # base case
    if issubclass(type(s), basestring):
        yield s
    # Recursive case
    else:
        for subelt in s:
            for elt in rrun(subelt):
                yield elt

In [105]:
test = [['1','2'], ['3', '4'], '5', '6']

[s for s in rrun(test)]

['1', '2', '3', '4', '5', '6']

In [15]:
def flatten(nested):
    try:
        try: nested + ''
        except TypeError: pass
        else: raise TypeError
        for sublist in nested:
            for element in flatten(sublist):
                yield element
    except TypeError:
        yield nested
        
print list(flatten(['foo', ['bar', ['baz']]]))

['foo', 'bar', 'baz']


In [109]:
list(flatten([1, 2, 3]))

[1, 2, 3]

In [41]:
[i for i in flatten([((j for j in xrange(i)) for i in xrange(5)), 9, 8])]

[0, 0, 1, 0, 1, 2, 0, 1, 2, 3, 9, 8]

In [112]:
def nested_map(root, func):
    """Takes in an arbitrarily nested iterator and traverses it
     deapth first to create generators at every level, where
     every element has a function applied to it.
     """
    try:
        # Don't unwind strings
        try:
            root + ''
        except TypeError:
            pass
        else:
            raise TypeError
        
        return (nested_map(elt, func) for elt in root)
        
    except TypeError:
        return func(root)

In [116]:
[i for i in nested_map(test, int)]

[<generator object <genexpr> at 0x7f20b18c0230>,
 <generator object <genexpr> at 0x7f20b18c0fa0>,
 5,
 6]

In [142]:
#! /usr/bin/env python2

"""
Idea behind the structure:

 String Level Transformations
   - Encodings
   - Substitutions
 Tokenize
 Sentence Level Transformations
   - Filters
   - Transformations
 Token Level Transformations
   - Filters
   - Substitutions
   - Additions
       - Append POS, lemmatize, etc

"""

import re
import string
# import itertools
from bs4 import BeautifulSoup as bs
from gensim.utils import any2unicode, any2utf8, decode_htmlentities, deaccent
from gensim.parsing import preprocessing as gprocessing
from candidate_classifier import utils


__author__ = 'Eric Lind'


# TODO:
# pre/postfilter substitutions
#  - What if the substitution you perform changes the string so that it would have been filtered?


class StringProcessor(object):
    def __init__(self,
                 string_transformer,
                 sentence_transformer=None,
                 token_transformer=None):
        """Autobots, roll out!"""
        self.str_transformer = string_transformer
        self.sent_transformer = sentence_transformer
        self.tok_transformer = token_transformer

    def __call__(self, s):
        # Process string
        processed = self.str_transformer(s)

        # TODO: Fix this and figure our contract
        # Sentences
        if self.sent_transformer is not None:
            sents = self.sent_transformer(s)

            for sent in sents:
                # Tokens
                if self.tok_transformer is not None:
                    sent = self.tok_transformer(sent)

                yield sent



    # def __call__(self, s):
    #     Yes, I know this is difficult to read.  But this is
    #     my library and I'll do as I please thank you very much.
    #     The contract of this reduce operation is essentially this:
    #     f([a, b, c], 's') ==> c(b(a('s')))
    #     The reason for the reversal is so that when a user specifies
    #     the function they want to call first, it can be first in the list.
        # return reduce(lambda x, y: y(x), self.transformers, s)


# TODO:
# Make a repr that shows the current filters/substitutions being used
# Add the available built-in filters/substitutions to the docstring
class TransformerABC(object):
    """Interface for applying transformations to a string/token"""

    re_type = type(re.compile(r''))
    ws_pattern = re.compile(r"white(?:-|_|\s)?space", re.I)
    non_ascii_pattern = re.compile(r"non(?:-|_|\s)?ascii", re.I)
    h_ents_pattern = re.compile(r"html(?:-|_|\s)?entities", re.I)
    # TODO: Make min_len an optional argument
    min_len = 3

    def __init__(self,
                 prefilter_substitutions=(),
                 postfilter_substitutions=(),
                 filters=(),
                 normalize_encoding=True,
                 flatten=True,  # TODO: Document
                 # Tokenization always occurs last.
                 # If you want to process the tokens, use another Transformer
                 # tokenizer must be callable
                 tokenizer=None,
                 **kwargs):
        self.filters = self._process_filters(filters)
        self.pre_substitutions = self._process_substitutions(prefilter_substitutions)
        self.post_substitutions = self._process_substitutions(postfilter_substitutions)
        self.normalize_encoding = normalize_encoding
        self.tokenizer = tokenizer

        # Set processor
        self.processor = self._flat_process if flatten else self._nested_process


    def __call__(self, s):
        # FIXME More documnetation for empty string behavior
        return (i for i in self.processor(s) if i)


    def _process_filters(self, filts):
        """
        Filters are functions that return booleans.  If a function
        resturns True (or some truthy value) for a given string,
        that string is filtered out.
        """
        t = []

        for f in filts:
            # Length
            if f in {'length', 'len'}:
                # Default length is 3
                # Create a closure
                def funcC(fil):
                    def func(s): return len(s) <= self.min_len
                    return func
                t.append(funcC(f))
                # t.append(lambda s: len(s) <= self.min_len)
            elif hasattr(f, '__getitem__'):
                try:
                    if f[0] in {'length', 'len', 'short'}:
                        t.append(lambda s: len(s) <= s[1])
                # FIXME: More thorough checks
                except (IndexError, TypeError):
                    pass

            # Patterns
            elif isinstance(f, self.re_type):
                # Create a closure for the current filter
                def funcC(fil):
                    def func(s): return re.search(fil, s)
                    return func
                t.append(funcC(f))
                # t.append(lambda s: re.search(f, s))

            # Non-Ascii
            elif re.search(self.non_ascii_pattern, str(f)):
                t.append(self._is_non_ascii)

            # Callable
            elif hasattr(f, '__call__'):
                t.append(f)

            # TODO:
            # html tags
            # punctuation

        return t


    def _process_substitutions(self, subs):
        t = []

        # Go through all subs and prepare/apply them.
        # The order is IMPORTANT.  It is expected that
        # the user knows what order they want!
        for sub in subs:
            # HTML
            if sub == 'html':
                t.append(lambda s: bs(s, 'lxml').get_text())

            # HTML entities
            # if sub in {'htmlentities', 'html entities', 'html_entities', 'html-entities'}:
            elif re.search(self.h_ents_pattern, str(sub)):
                t.append(decode_htmlentities)

            # Daccent
            elif sub == 'deaccent':
                t.append(deaccent)

            # Punctuation
            elif sub in {'punct', 'punctuation', 'puncts'}:
                t.append(gprocessing.strip_punctuation)

            # Case
            elif sub == 'lower':
                t.append(string.lower)
            elif sub == 'upper':
                t.append(string.upper)

            # Whitespace
            elif re.search(self.ws_pattern, str(sub)):
                t.append(gprocessing.strip_multiple_whitespaces)
            elif sub == 'strip':
                t.append(string.strip)

            # callable
            elif hasattr(sub, '__call__'):
                t.append(sub)

            # patterns
            elif isinstance(sub, self.re_type):
                # TODO: Make more elegant
                def funcC(sub):
                    def func(s): return sub.sub('', s)
                    return func
                t.append(funcC(sub))
                # t.append(lambda s: sub.sub('', s))
            else:
                try:
                    if isinstance(sub[0], self.re_type):
                        def funcC(sub):
                            # FIXME: Use compiled pattern
                            # def func(s): sub[0].sub(sub[1], s)
                            def func(s): return re.sub(sub[0], sub[1], s)
                            return func
                        t.append(funcC(sub))
                        # t.append(lambda s: re.sub(sub[0], sub[1], s))
                except IndexError:
                    pass

        return t

    @staticmethod
    def _is_non_ascii(s):
        """Returns True if the string contains non-ascii characters
        Fastest way to check for non-ascii:
        http://stackoverflow.com/questions/196345/how-to-check-if-a-string-in-python-is-in-ascii
        """
        try:
            s.decode('ascii')
        except UnicodeDecodeError:
            return True
        else:
            return False

    def _flat_process(self, s):
        """Takes in either an iterator or a string and yields strings.
        Calling this on a nested list will result in a flattened list.
        """
        for elt in utils.flatten(s):
            # print elt
            # Tokenization may add another level of nesting
            for t in utils.flatten(self._process(elt)):
                # print t
                yield t

    def _nested_process(self, s):
        return utils.nested_map(s, self._process)

    def _process(self, s):
        # TODO: Clarify contract return value for filtered strings
        # TODO: Add docs for: if a filter evaluates as False the string is discarded

        # Skip empty strings:
        if len(s) == 0:
            return s

        # Normalize the encoding before anything else
        if self.normalize_encoding:
            s = any2unicode(s)

        # Apply sub, filter, sub
        for sub in self.pre_substitutions:
            s = sub(s)

        # If any of the filters return True, filter the string
        if any(f(s) for f in self.filters):
            return ''

        for sub in self.post_substitutions:
            s = sub(s)

        # Tokenize last.  If you want to process the tokens use another
        # Transformer
        if self.tokenizer:
            s = self.tokenizer(s)

        return s






class StringTransformer(TransformerABC):
    def __init__(self,
                 prefilter_substitutions=(),
                 postfilter_substitutions=(),
                 filters=()):
        super(StringTransformer, self).__init__(prefilter_substitutions,
                                                postfilter_substitutions,
                                                filters)

    def __call__(self, s):
        # Normalize encoding
        # TODO: Find a more robust function for this (e.g. from another library)
        s = any2unicode(s)

        return self._process(s)



class SentenceTransformer(TransformerABC):
    def __init__(self,
                 prefilter_substitutions=(),
                 postfilter_substitutions=(),
                 filters=(),
                 sent_tokenizer=None):
        super(SentenceTransformer, self).__init__(prefilter_substitutions,
                                                  postfilter_substitutions,
                                                  filters)
        # If no tokenizer is specified, it is assumed that
        # this will be called on a pre-tokenized list of sentences
        self.tokenizer = sent_tokenizer

    def __call__(self, s):
        """Takes in a string and yields sentences"""
        if self.tokenizer is not None:
            sents = self.tokenizer(s)
        else:
            sents = s

        for sent in sents:
            yield self._process(sent)



# TODO:
# - Stopwords
# - Punctuation
# is non-ascii
# is numeric
# is url
# is email address
class TokenTransformer(TransformerABC):
    def __init__(self,
                 prefilter_substitutions=(),
                 postfilter_substitutions=(),
                 filters=(),
                 tokenizer=None):
        super(TokenTransformer, self).__init__(prefilter_substitutions,
                                               postfilter_substitutions,
                                               filters)
        # self.filters.extend(self._process_token_filters(filters))
        # self.pre_substitutions.extend(self._process_token_substitutions(prefilter_substitutions))
        # self.post_substitutions.extend(self._process_token_substitutions(postfilter_substitutions))

        # If no tokenizer is specified, it is assumed that
        # this will be called on a pre-tokenized list of words
        self.tokenizer = tokenizer

    def _process_token_filters(self, filters):
        # Stopwords
        # if f in {'stop', 'stops', 'stopwords'}:
        #     pass
        pass


    def _process_token_substitutions(self, substitutions):
        pass

    # def __call__(self, sent):
    #     """Takes in a

In [6]:
def foo(docs):
    if isinstance(docs, basestring):
        return docs.split()
    else:
        return bar(docs)

def bar(docs):
    for doc in docs:
        yield doc.split()
        
        
docs = ["This is a test.", "A what?"]
d1 = "A test!"

[foo(d) for d  in docs]

[['This', 'is', 'a', 'test.'], ['A', 'what?']]

In [15]:
isinstance('foo', (str, int))

True